# 🏭 THE CONVERSATION FACTORY: Synthetic Data Generation

**Mission:** Turn raw Indonesian Laws into high-quality QA pairs for Nuzantara.
**Model:** `Llama-3-8B-Instruct` (via Unsloth/HuggingFace) - Running on Colab GPU.
**Output:** `synthetic_gold_dataset.json` (User Question + Ideal Answer + Reasoning).

## 🧠 Strategy
We will feed chunks of law text to Llama 3 and ask it to hallucinate *realistic* user scenarios and *perfect* legal answers.

In [ ]:
# 🚀 Install Dependencies (Unsloth for fast inference)
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
# 📥 Setup & Mount Drive
from google.colab import drive
import os
import json
import glob
from tqdm.notebook import tqdm

drive.mount('/content/drive')

# Paths
RAW_TEXT_DIR = "/content/drive/MyDrive/nuzantara_laws_text" # Where we saved TXTs in Phase 1
OUTPUT_FILE = "/content/drive/MyDrive/synthetic_gold_dataset.json"

# Check if text exists
if not os.path.exists(RAW_TEXT_DIR):
    print(f"⚠️ Warning: {RAW_TEXT_DIR} not found. Did you run the Law Eater with text export?")
    # Fallback: Create dummy dir for testing
    os.makedirs(RAW_TEXT_DIR, exist_ok=True)
else:
    print(f"✅ Found Raw Text Directory: {RAW_TEXT_DIR}")

In [ ]:
# 🦙 Load Llama 3 (4-bit Quantized for Colab Free Tier)
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

print("✅ Llama 3 Loaded! Ready to generate.")

In [ ]:
# 🎭 The Prompt Template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You are an expert Indonesian Legal AI trainer. Your task is to generate a synthetic QA pair based on the provided law text.
1. **User Question**: Create a realistic question a business owner or foreigner might ask about this law. It can be slightly informal.
2. **Ideal Answer**: Write the perfect, legally accurate answer based ONLY on the provided text. Cite the specific Article (Pasal) if mentioned.
3. **Category**: Classify as 'Visa', 'Tax', 'Corporate', or 'General'.

Output valid JSON format: {{"question": "...", "answer": "...", "category": "..."}}

### Input:
{}

### Response:
"""

def generate_qa(text_chunk):
    inputs = tokenizer(
        [alpaca_prompt.format(text_chunk)], return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
    response = tokenizer.batch_decode(outputs)[0]
    
    # Extract JSON from response (naive parsing)
    try:
        json_str = response.split("### Response:")[1].strip()
        # Cleanup markdown code blocks if present
        if "```json" in json_str:
            json_str = json_str.split("```json")[1].split("```")[0]
        elif "```" in json_str:
            json_str = json_str.split("```")[1].split("```")[0]
            
        return json.loads(json_str)
    except:
        return None

In [ ]:
# 🏭 Production Line
import random

files = glob.glob(f"{RAW_TEXT_DIR}/*.txt")
dataset = []

print(f"📚 Found {len(files)} text files. Starting generation...")

# Sample random files to avoid processing everything in one go (for demo)
# In production, remove the slice [:10]
for file_path in tqdm(files[:10], desc="Generating Data"):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
        
    # Take a random chunk of 1000-2000 chars to generate a question from
    if len(text) > 2000:
        start = random.randint(0, len(text) - 2000)
        chunk = text[start:start+2000]
    else:
        chunk = text
        
    qa_pair = generate_qa(chunk)
    if qa_pair:
        qa_pair['source_file'] = os.path.basename(file_path)
        dataset.append(qa_pair)
        print(f"✅ Generated: {qa_pair['question'][:50]}...")

# Save Dataset
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=2)
    
print(f"🎉 Generation Complete! Saved {len(dataset)} pairs to {OUTPUT_FILE}")